In [2]:
import os
import sys
import time
import json
import matplotlib.pyplot as plt
sys.path.append("/home/ziniuw/zero-shot-cost-estimation")
from models.zero_shot_models.specific_models.model import zero_shot_models
from cross_db_benchmark.benchmark_tools.database import DatabaseSystem
from models.training.train import train_default, train_readout_hyperparams

Using backend: pytorch


In [7]:
import json
from cross_db_benchmark.benchmark_tools.utils import load_json


def add_num_tables(src_workloads):
    for src in src_workloads:
        try:
            run = load_json(src, namespace=False)
        except JSONDecodeError:
            raise ValueError(f"Error reading {src}")

        all_num_tables = []
        for q_id, p in enumerate(run["parsed_plans"]):
            tables = get_tables(p)
            assert len(tables) != 0
            p["num_tables"] = len(tables)
            all_num_tables.append(len(tables))

        with open(src, 'w') as outfile:
            json.dump(run, outfile)
        print(f"done adding num_tables for {src}, with max num_table {max(all_num_tables)} min {min(all_num_tables)}")


def get_tables(plan):
    tables = set()
    if 'table' in plan["plan_parameters"]:
        tables.add(plan["plan_parameters"]['table'])

    for c in plan["children"]:
        c_tables = get_tables(c)
        tables.update(c_tables)

    return tables

In [14]:
workload_runs = ["../zero-shot-data/runs/parsed_plans/airline/complex_workload_200k_s1_c8220.json", 
                      "../zero-shot-data/runs/parsed_plans/airline/workload_100k_s1_c8220.json", 
                      "../zero-shot-data/runs/parsed_plans/ssb/complex_workload_200k_s1_c8220.json", 
                      "../zero-shot-data/runs/parsed_plans/ssb/workload_100k_s1_c8220.json", 
                      "../zero-shot-data/runs/parsed_plans/tpc_h/complex_workload_200k_s1_c8220.json",  
                      "../zero-shot-data/runs/parsed_plans/walmart/complex_workload_200k_s1_c8220.json", 
                      "../zero-shot-data/runs/parsed_plans/walmart/workload_100k_s1_c8220.json", 
                      "../zero-shot-data/runs/parsed_plans/financial/complex_workload_200k_s1_c8220.json", 
                      "../zero-shot-data/runs/parsed_plans/financial/workload_100k_s1_c8220.json", 
                      "../zero-shot-data/runs/parsed_plans/basketball/complex_workload_200k_s1_c8220.json", 
                      "../zero-shot-data/runs/parsed_plans/basketball/workload_100k_s1_c8220.json", 
                      "../zero-shot-data/runs/parsed_plans/accidents/complex_workload_200k_s1_c8220.json", 
                      "../zero-shot-data/runs/parsed_plans/accidents/workload_100k_s1_c8220.json", 
                      "../zero-shot-data/runs/parsed_plans/movielens/complex_workload_200k_s1_c8220.json", 
                      "../zero-shot-data/runs/parsed_plans/movielens/workload_100k_s1_c8220.json", 
                      "../zero-shot-data/runs/parsed_plans/baseball/complex_workload_200k_s1_c8220.json", 
                      "../zero-shot-data/runs/parsed_plans/baseball/workload_100k_s1_c8220.json", 
                      "../zero-shot-data/runs/parsed_plans/hepatitis/complex_workload_200k_s1_c8220.json", 
                      "../zero-shot-data/runs/parsed_plans/hepatitis/workload_100k_s1_c8220.json", 
                      "../zero-shot-data/runs/parsed_plans/tournament/complex_workload_200k_s1_c8220.json", 
                      "../zero-shot-data/runs/parsed_plans/tournament/workload_100k_s1_c8220.json", 
                      "../zero-shot-data/runs/parsed_plans/credit/complex_workload_200k_s1_c8220.json", 
                      "../zero-shot-data/runs/parsed_plans/credit/workload_100k_s1_c8220.json", 
                      "../zero-shot-data/runs/parsed_plans/employee/complex_workload_200k_s1_c8220.json", 
                      "../zero-shot-data/runs/parsed_plans/employee/workload_100k_s1_c8220.json", 
                      "../zero-shot-data/runs/parsed_plans/consumer/complex_workload_200k_s1_c8220.json", 
                      "../zero-shot-data/runs/parsed_plans/consumer/workload_100k_s1_c8220.json", 
                      "../zero-shot-data/runs/parsed_plans/geneea/complex_workload_200k_s1_c8220.json", 
                      "../zero-shot-data/runs/parsed_plans/geneea/workload_100k_s1_c8220.json", 
                      "../zero-shot-data/runs/parsed_plans/genome/complex_workload_200k_s1_c8220.json", 
                      "../zero-shot-data/runs/parsed_plans/genome/workload_100k_s1_c8220.json", 
                      "../zero-shot-data/runs/parsed_plans/carcinogenesis/complex_workload_200k_s1_c8220.json", 
                      "../zero-shot-data/runs/parsed_plans/carcinogenesis/workload_100k_s1_c8220.json", 
                      "../zero-shot-data/runs/parsed_plans/seznam/complex_workload_200k_s1_c8220.json", 
                      "../zero-shot-data/runs/parsed_plans/seznam/workload_100k_s1_c8220.json", 
                      "../zero-shot-data/runs/parsed_plans/fhnk/complex_workload_200k_s1_c8220.json", 
                      "../zero-shot-data/runs/parsed_plans/fhnk/workload_100k_s1_c8220.json",
                "../zero-shot-data/runs/parsed_plans/imdb_full/complex_workload_400k_s4_c8220.json",
                "../zero-shot-data/runs/parsed_plans/imdb_full/complex_workload_400k_s5_c8220.json",
                "../zero-shot-data/runs/parsed_plans/imdb_full/complex_workload_400k_s6_c8220.json",
                "../zero-shot-data/runs/parsed_plans/imdb_full/complex_queries_training_50k.json",
                "../zero-shot-data/runs/parsed_plans/imdb_full/job_full_c8220.json",
                "../zero-shot-data/runs/parsed_plans/MSCN_augmented/complex_queries_testing_2k.json",
                "../zero-shot-data/runs/parsed_plans/MSCN_augmented/complex_queries_testing_2k_tuned_0.5.json",
                "../zero-shot-data/runs/parsed_plans/MSCN_augmented/complex_queries_testing_2k_tuned_0.9.json",]
add_num_tables(workload_runs)

done adding num_tables for /flash1/ziniuw/zero-shot-data/runs/parsed_plans/imdb_full/complex_workload_400k_s4_c8220.json, with max num_table 5 min 1
